# Limpieza y visualización de datos

En esta práctica vamos a ver diferentes formas de trabajar con datos nulos:
- eliminando las filas con valores NA
- sustituyendo valores NA por la media
- sustituyendo valores NA por la mediana

Para esta práctica, vamos a referirnos a la infección por el virus de la hepatitis C (HCV por sus siglas en inglés) que es un problema de salud que afecta a más de 100 millones de personas en el mundo. No obstante, y a pesar de la elevada cifra de afectados, sólo una parte de ellos presentará problemas graves derivados de la persistencia del virus. La infección aguda por el HCV es habitualmente asintomática y, por tanto, difícil de reconocer.

La prueba del virus de la hepatitis C consiste en un análisis de sangre que detecta material genético del virus que causa la hepatitis o las proteínas que produce el cuerpo contra el HCV. Para ello se utiliza la reacción en cadena de la polimerasa (PCR).

In [ ]:
#Se cargan las librerías iniciales
library(ggplot2)
library(data.table)
library(tidyverse)

# Datos
Cargamos los datos directamente de la web https://archive.ics.uci.edu/ml/datasets/HCV+data.
El conjunto de datos consiste en 615 registros de pacientes con 14 variables:

- **X**: número del paciente
- **Category**: Distingue donantes sanos de personas afectadas y los grados de afectación. Categórica. 5 valores diferentes.
- **Age**: Edad del paciente. Numérica.
- **Sex**: Sexo del paciente. Categórica. 2 valores diferentes.
- **ALB**: Nivel de albúmina en sangre. Numérica.
- **ALP**: Nivel de fosfatasa alcalina en sangre. Numérica.
- **ALT**: Nivel de transaminasa alcalina en sangre. Numérica.
- **AST**: Nivel de aspartato de aminotransferasa en sangre. Numérica.
- **BIL**: Nivel de bilirrubina en sangre. Numérica.
- **CHE**: Nivel de colinesterasa en sangre. Numérica.
- **CHOL**: Nivel de colesterol en sangre. Numérica.
- **CREA**: Nivel de creatinina en sangre. Numérica.
- **GGT**: Nivel de gamma glutamil transferasa en sangre. Numérica.
- **PROT**: Nivel de proteína en sangre. Numérica.

In [ ]:
# Cargamos datos directamente de la web en la variable datos

datos <- read.csv("http://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv")

In [ ]:
# Comprobamos el número de datos y vemos el encabezado y
# las primeras filas
dim(datos)
head(datos)

In [ ]:
# Vemos un resumen del tipo de variables
str(datos)

In [ ]:
# Vemos un resumen de los datos
summary(datos)

Vemos valores NA en algunas variables (ALB, ALP...). Podemos contarlos por columnas 

In [ ]:
#Contar el total de NAs en la base de datos
sum(is.na(datos))

In [ ]:
# Contar número de nulos por columna
sapply(datos, function(x) sum(is.na(x)))

Una opción muy interesante es visualizar cuántos y dónde tenemos valores nulos con la función `vis_miss` del paquete **visdat**.

In [ ]:
# Si no está instalada:
# Utilice este código: install.packages("visdat") para instalar la librería
library(visdat)
vis_miss(datos)

Tenemos varias opciones para proceder con los valores nulos:
- Opción 1: Eliminar todas las filas con datos nulos
- Opción 2: eliminar algunas filas con datos NA
- Opción 3: Imputación con la media
- Opción 4: imputación con la mediana

# Opción 1: Eliminar todas las filas con datos nulos 

Hemos visto que únicamente un porcentaje muy pequeño del total de los datos son NA. En esta opción, con la función `na.omit`, eliminamos las filas con datos NA.

In [ ]:
# Eliminamos las filas con algún dato NA
datos1 <-  na.omit(datos)
sapply(datos1, function(x) sum(is.na(x)))

In [ ]:
# Comparamos el número de datos original y el obtenido
dim(datos)
dim(datos1)

# Opción 2: eliminar algunas filas con datos NA

En algunos casos nos puede interesar eliminar únicamente algunas filas con datos NA. Para ello, primero localizamos las filas con datos nulos y procedemos a eliminarlas una por una.

In [ ]:
# Localizar filas con NA función complete.cases
# El símbolo ! indica que no muestre los que tienen todos los datos, es decir,
# únicamente va a mostrar los que tienen valores NA
head(datos[!complete.cases(datos), ])

In [ ]:
# Listado de filas con NA almacenados en with_na
with_na <- datos[!complete.cases(datos), ]
with_na

In [ ]:
# Eliminamos (como ejemplo) la fila 122 y almacenamos en datos2
datos2 <- datos[-122,]
# comprobamos que la hemos eliminado
with_na2 <- datos2[!complete.cases(datos2), ]
with_na2

# Opción 3: Imputación con la media

Evidentemente, esta opción 3 únicamente es válida para variables con valores numéricos.
En este caso se sustituye cada valor NA por el valor medio de la variable (columna) a la que pertenece.

In [ ]:
# Reemplazo de NA por la media de la columna correspondiente
datos_media <- data.frame(
    sapply(
        datos,
        function(x) ifelse(is.na(x),
            mean(x, na.rm = TRUE),
            x)))
# Comprobamos que ya no hay NA
        sapply(datos_media, function(x) sum(is.na(x)))

# Opción 4: imputación con la mediana

Recordamos que para el cálculo de la mediana, sólo influyen los valores centrales de la distribución. Además, la mediana es insensible a valores extremos (la media es muy sensible a valores extremos). 
De este modo, habrá casos en los que nos interese sustituir los valores nulos por la mediana de la variable correspondiente.

In [ ]:
# Reemplazo de Na por la media de la columna correspondiente
datos_mediana <- data.frame(
    sapply(
        datos,
        function(x) ifelse(is.na(x),
            median(x, na.rm = TRUE),
            x)))
# Comprobamos que ya no hay NA
        sapply(datos_mediana, function(x) sum(is.na(x)))

# Limpieza de datos
Comenzamos transformando las variables categóricas en factores en el conjunto de datos original.

In [ ]:
datos$Sex <- as.factor(datos$Sex)
datos$Category <- as.factor(datos$Category)

La variable `Category` del dataset original puede tener los siguientes valores:

In [ ]:
levels(datos$Category)

Dado que las categorías “0=Blood Donor” y “0s=suspect Blood Donor” se refieren a personas no infectadas por HCV, se cambian esas categorías por “Donor”. El resto de categorías se renombran como “Hepatitis”, “Fibrosis”, “Cirrhosis”

In [ ]:
levels(datos$Category) <- c("Donor", "Donor", "Hepatitis", "Fibrosis", "Cirrhosis")
datos <- as.data.table(datos)
levels(datos$Category)

# Visualización de datos
Procedemos a realizar un estudio preliminar de la distribución de datos de forma visual.
Vemos cómo es la distribución de la enfermedad por edad.

In [ ]:
p <- ggplot(data = datos[Category!="Donor"], aes(x = Age)) +
     geom_histogram( aes(fill = Category)) +
     scale_fill_brewer(palette="Dark2") +
     theme(plot.title = element_text(face = "bold"))
q <- p + facet_grid(cols=vars(Category))
cowplot::plot_grid(p, q, ncol = 1) 

Veamos cómo es cada parámetro sanguíneo por categoría:

In [ ]:
hcvdat <- datos
geomBoxplotGraph <- function(data, x, y, title){
  g <- ggplot(data, aes(x, y, colour = x)) +
    geom_boxplot() +
    labs(title = title) +
    scale_fill_brewer(palette="Dark2") +
    theme(axis.text.x = element_text(angle = 30))
  return(g)
}

g.ALB <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALB, "Albúmina (ALB)")
g.ALP <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALP, "Fosfatasa Alcalina (ALP)")
g.ALT <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALT, "Transaminasa Alcalina (ALT)")
g.AST <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$AST, "Aspartato de Aminotransferasa (AST)")
g.BIL <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$BIL, "Bilirrubina (BIL)")
g.CHE <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$CHE, "Colinesterasa (CHE)")
g.CHOL <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALB, "Colesterol (CHOL)")
g.CREA <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALB, "Creatinina (CREA)")
g.GGT <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALB, "Gamma Glutamil Transferasa (GGT)")
g.PROT <- geomBoxplotGraph(hcvdat, hcvdat$Category, hcvdat$ALB, "Proteína (PROT)")
cowplot::plot_grid(g.ALB, g.ALP , g.ALT, g.AST, g.BIL, g.CHE, g.CHOL, g.CREA, g.GGT, g.PROT, ncol=2)

Hemos obtenido algún error debido a que estamos utilizando los datos originales sin haber hecho tratamiento alguno con los datos nulos.
A la vista de las gráficas puede parecer que, en algunos casos, se puede aislar alguna categoría de las otras en función de los valores de la variable. Vamos a ver la nube de puntos para comprobar si eso es posible o no.

In [ ]:
# Se crea un data table alargando hcvdat de manera que la columna 
# _Value_ contiene los valores de cada parámetro sanguíneo 
# y los nombres de éstas columnas pasan a ser categorías.
hcvdat_pt <- select(hcvdat, X, ALB:PROT) %>% gather(Parameter, Value, -X)
hcvdat_pt <- merge.data.table(hcvdat_pt, hcvdat[,.(X , Category)], by = "X")

f <- ggplot(hcvdat_pt, aes(Parameter, Value) ) +
  geom_point(aes(colour = Category, shape = Category)) +
  labs(title = "Parámetros sanguíneos") +
  scale_fill_brewer(palette="Dark2") +
  theme(axis.text.x = element_text(angle = 30))
f

Vemos que se solapan los rangos de los valores que toma cada categoría en cada variable y, por tanto, no es posible aislar los puntos en cada caso en varios tramos.
Al igual que en la obtención de los gráficos anteriores, nos ha aparecido un mensaje de error debido a la presencia de valores nulos; por ello es conveniente eliminarlos mediante alguna de las opciones descritas en esta práctica.

**Ejercicio**: obtenga las gráficas anteriores cambiando el conjunto de datos original `datos` por los datos obtenidos con las opciones anteriores:

- `datos 1` correspondiente a la Opción 1
- `datos_media` correspondiente a la Opción 3
- `datos_mediana` correspondiente a la Opción 4

Recuerde realizar la limpieza de datos previamente.